In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import yaml

In [3]:
# --- Configuration ---
VIDEOS_PATH = '../Synthetic Meteor Video Generation/output_videos'
GT_CSV_PATH = '../Synthetic Meteor Video Generation/ground_truth/dataset_labels.csv'
YOLO_DATASET_DIR = 'meteor_yolo_dataset'

In [ ]:
# # --- 1. Create YOLO Directory Structure ---
# print("Creating YOLOv12 directory structure...")
# for split in ['train', 'val']:
#     os.makedirs(os.path.join(YOLO_DATASET_DIR, 'images', split), exist_ok=True)
#     os.makedirs(os.path.join(YOLO_DATASET_DIR, 'labels', split), exist_ok=True)

# # --- 2. Load Ground Truth and Split the Data ---
# try:
#     df = pd.read_csv(GT_CSV_PATH)
# except FileNotFoundError:
#     print(f"ERROR: Ground truth file not found at '{GT_CSV_PATH}'")
#     exit()

# train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
# print(f"Dataset split: {len(train_df)} training videos, {len(val_df)} validation videos.")
# print("Each video will contribute multiple frames (positive and negative samples).")

Creating YOLOv12 directory structure...
Dataset split: 149015 training videos, 37254 validation videos.
Each video will contribute multiple frames (positive and negative samples).


In [ ]:
# # --- 3. Process Both Splits (Train and Val) ---
# for split_name, split_df in [('train', train_df), ('val', val_df)]:
#     print(f"\nProcessing '{split_name}' split...")
    
#     image_dir = os.path.join(YOLO_DATASET_DIR, 'images', split_name)
#     label_dir = os.path.join(YOLO_DATASET_DIR, 'labels', split_name)

#     for index, row in tqdm(split_df.iterrows(), total=len(split_df)):
#         video_filename = row['video_file']
#         video_path = os.path.join(VIDEOS_PATH, video_filename)

#         if not os.path.exists(video_path):
#             continue

#         cap = cv2.VideoCapture(video_path)
#         total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#         if total_frames < 4: # Need at least a few frames to make this meaningful
#             cap.release()
#             continue

#         base_filename = os.path.splitext(video_filename)[0]

#         # --- a) Handle Negative Sample (Frame 0) ---
#         cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
#         ret, frame = cap.read()
#         if ret:
#             image_filename = f"{base_filename}_frame_0_neg.jpg"
#             image_filepath = os.path.join(image_dir, image_filename)
#             cv2.imwrite(image_filepath, frame)
#             # No label file is created for this negative sample.

#         # --- b) Handle Positive Samples ---
#         # Calculate the single bounding box for the full streak
#         x_coords = [row['start_x'], row['end_x']]; y_coords = [row['start_y'], row['end_y']]
#         box_w = abs(x_coords[1] - x_coords[0]); box_h = abs(y_coords[1] - y_coords[0])
#         x_center = min(x_coords) + box_w / 2; y_center = min(y_coords) + box_h / 2
#         x_center_norm = x_center / 1024; y_center_norm = y_center / 1024
#         box_w_norm = box_w / 1024; box_h_norm = box_h / 1024
#         yolo_label_string = f"0 {x_center_norm:.6f} {y_center_norm:.6f} {box_w_norm:.6f} {box_h_norm:.6f}"

#         # Define the indices for our positive frames, including the last one
#         positive_frame_indices = {
#             total_frames // 2, # A frame from the middle
#             total_frames - 1   # The final, complete frame
#         }

#         for i in sorted(list(positive_frame_indices)): # sorted() ensures a consistent order
#             cap.set(cv2.CAP_PROP_POS_FRAMES, i)
#             ret, frame = cap.read()
#             if ret:
#                 image_filename = f"{base_filename}_frame_{i}_pos.jpg"
#                 image_filepath = os.path.join(image_dir, image_filename)
#                 cv2.imwrite(image_filepath, frame)
                
#                 label_filename = f"{base_filename}_frame_{i}_pos.txt"
#                 label_filepath = os.path.join(label_dir, label_filename)
#                 with open(label_filepath, 'w') as f:
#                     f.write(yolo_label_string)
        
#         cap.release()


Processing 'train' split...


  0%|          | 0/149015 [00:00<?, ?it/s]

100%|██████████| 149015/149015 [9:52:41<00:00,  4.19it/s]   



Processing 'val' split...


100%|██████████| 37254/37254 [1:56:51<00:00,  5.31it/s]   


In [ ]:
# # --- 4. Create the data.yaml file ---
# yaml_path = os.path.join(YOLO_DATASET_DIR, 'data.yaml')
# yaml_content = {
#     'path': os.path.abspath(YOLO_DATASET_DIR),
#     'train': os.path.join('images', 'train'),
#     'val': os.path.join('images', 'val'),
#     'nc': 1,
#     'names': ['meteor']
# }
# with open(yaml_path, 'w') as f:
#     yaml.dump(yaml_content, f, sort_keys=False)

# print(f"\n✅ YOLOv8 dataset preparation complete!")
# print(f"Your dataset is ready for training in: '{YOLO_DATASET_DIR}'")
# print("\nTo start training, run the following command:")
# print(f"yolo task=detect mode=train model=yolov12n.pt data='{os.path.abspath(yaml_path)}' epochs=50 imgsz=640")


✅ YOLOv8 dataset preparation complete!
Your dataset is ready for training in: 'meteor_yolo_dataset'

To start training, run the following command:
yolo task=detect mode=train model=yolov12n.pt data='d:\My Laptop\GU\Research Papers\Meteor Detection and Trajectory Estimation Using Object Detection and Tracking\Training a Meteor Detector on Synthetic Data\meteor_yolo_dataset\data.yaml' epochs=50 imgsz=640


In [4]:
os.listdir(YOLO_DATASET_DIR)  # To ensure the directory is created and populated

['data.yaml', 'images', 'labels']

In [5]:
from ultralytics import YOLO
# Optionally, you can train the model directly after preparation
# --- This should be the variable from the previous script ---
# It points to the 'data.yaml' file inside your 'meteor_yolo_dataset' folder.
# We'll define it explicitly here for clarity.
yaml_path = os.path.abspath('meteor_yolo_dataset/data.yaml')

model = YOLO('D:\My Laptop\Me\Programming\Machine Learning\Computer Vision\YOLO Models\YOLOv12\Detection\yolo12n.pt')

# 2. Train the model
# We start with 3 epochs as a quick test.
results = model.train(
    data=yaml_path, 
    epochs=3, 
    imgsz=640, 
    name='meteor_detection_test_run', # Give it a unique name
    project='yolo_training_runs',      # All runs will be saved here
    exist_ok=True                      # Allows you to re-run without errors
)
print("\n✅ Training complete!")
print(f"Results saved to: {results.save_dir}")
print("\nTo view results, check the 'yolo_training_runs/meteor_detection_test_run' folder.")


New https://pypi.org/project/ultralytics/8.3.152 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.149  Python-3.10.0 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=d:\My Laptop\GU\Research Papers\Meteor Detection and Trajectory Estimation Using Object Detection and Tracking\Training a Meteor Detector on Synthetic Data\meteor_yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, ma

train: Scanning D:\My Laptop\GU\Research Papers\Meteor Detection and Trajectory Estimation Using Object Detection and Tracking\Training a Meteor Detector on Synthetic Data\meteor_yolo_dataset\labels\train.cache... 297912 images, 148956 backgrounds, 0 corrupt: 100%|██████████| 446868/446868 [00:00<?, ?it/s]
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 0.30.2 ms, read: 0.40.3 MB/s, size: 42.5 KB)


val: Scanning D:\My Laptop\GU\Research Papers\Meteor Detection and Trajectory Estimation Using Object Detection and Tracking\Training a Meteor Detector on Synthetic Data\meteor_yolo_dataset\labels\val.cache... 74490 images, 37245 backgrounds, 0 corrupt: 100%|██████████| 111735/111735 [00:00<?, ?it/s]
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to yolo_training_runs\meteor_detection_test_run\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.0005), 119 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to yolo_training_runs\meteor_detection_test_run
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G     0.8852      1.218       1.02          5        640: 100%|██████████| 27930/27930 [54:15:35<00:00,  6.99s/it]       
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3492/3492 [4:39:12<00:00,  4.80s/it]  


                   all     111735      74490      0.945      0.918      0.966       0.91

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/3         0G     0.6672     0.5686     0.9393          9        640:  34%|███▎      | 9417/27930 [18:18:18<35:59:11,  7.00s/it]  


KeyboardInterrupt: 

# Test on different dataset

In [ ]:
test_yaml_path = 'Test Data\Meteor Shower Detection.v1i.yolov8\data.yaml'
model = YOLO('yolo_training_runs/meteor_detection_test_run/weights/best.pt')

model.val(
    data=test_yaml_path, 
    project='yolo_validation_runs',
)